In [4]:
import pulp
import pandas
import numpy as np

In [5]:
Insumos={"Damasco","Durazno","Azucar"}
Mermeladas={"tipo1","tipo2","tipo3"}
         
Disponibilidad={"Damasco":7,"Durazno":6,"Azucar":99}
Costo={"Damasco":1000,"Durazno":1200,"Azucar":400}
Req={"tipo1":{"Damasco":0.7,"Durazno":0,"Azucar":0.3},
                "tipo2":{"Damasco":0.1,"Durazno":0.5,"Azucar":0.4},
                "tipo3":{"Damasco":0.5,"Durazno":0.5,"Azucar":0}}
Precio={"tipo1":2000,"tipo2":2200,"tipo3":3000}
Demanda={"tipo1":5,"tipo2":7,"tipo3":10}

In [6]:
prob = pulp.LpProblem("Ejemplo1", pulp.LpMaximize)

x = pulp.LpVariable.dicts("CantidadMermelada", (m for m in Mermeladas), lowBound=0, cat='Integer')
a = pulp.LpVariable.dicts("Insumos", (i for i in Insumos), lowBound=0, cat='Integer')


In [7]:
prob += pulp.lpSum([x[m]*Precio[m] for m in Mermeladas]) - pulp.lpSum([a[i]*Costo[i] for i in Insumos])


In [8]:
for i in Insumos:
    prob += pulp.lpSum([x[m]*Req[m][i] for m in Mermeladas])== a[i]

for i in Insumos:
    prob += a[i]<= Disponibilidad[i]

for m in Mermeladas:
    prob += x[m]<= Demanda[m]


In [9]:
# Choose solver, and set it to problem, and build the Gurobi model
solver = pulp.GUROBI()
prob.setSolver(solver)
prob.solver.buildSolverModel(prob)

# Solve model
prob.solve()


Using license file /home/maxi/anaconda3/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 9 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x266adb8c
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [4e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 0 columns
Presolve time: 0.01s
Presolved: 3 rows, 6 columns, 10 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 2.500000e+04, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 25000.0000    0    2   -0.00000 25000.0000      -     -    0s
H    0     0                    20000.000000 25000.00

1

In [11]:
for i in prob.variables():
    print(i, i.varValue)

CantidadMermelada_tipo1 4.0
CantidadMermelada_tipo2 7.0
CantidadMermelada_tipo3 5.0
Insumos_Azucar 4.0
Insumos_Damasco 6.0
Insumos_Durazno 6.0


In [12]:
%matplotlib widget

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

def plot_implicit(ax, fn, bbox=(-2.5,2.5)):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    xmin, xmax, ymin, ymax, zmin, zmax = bbox*3


    A = np.linspace(xmin, xmax, 100) # resolution of the contour
    B = np.linspace(xmin, xmax, 15) # number of slices
    A1,A2 = np.meshgrid(A,A) # grid on which the contour is plotted
    try:
        for z in B: # plot contours in the XY plane
            X,Y = A1,A2
            Z = fn(X,Y,z)
            cset = ax.contour(X, Y, Z+z, [z], zdir='z')
            # [z] defines the only level to plot for this contour for this value of z
    except TypeError:
        pass
    
    try:
        for y in B: # plot contours in the XZ plane
            X,Z = A1,A2
            Y = fn(X,y,Z)
            cset = ax.contour(X, Y+y, Z, [y], zdir='y')
    except TypeError:
        pass    
    try:
        for x in B: # plot contours in the YZ plane
            Y,Z = A1,A2
            X = fn(x,Y,Z)
            cset = ax.contour(X+x, Y, Z, [x], zdir='x')
    except TypeError:
        pass

    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)
    return ax
    

In [13]:
fig = plt.figure()
fig.canvas.layout.width = '7in'
ax = fig.add_subplot(111, projection='3d')
bbox=(0,0.75)

def demand(x,y,z):
    c=1
    return x+y+z-c

def R1(x,y,z):
    
    return x-0.7


def R2(x,y,z):
    
    return y-0.1


def R3(x,y,z):
    
    return z-0.5

ax=plot_implicit(ax,R1,bbox)
ax=plot_implicit(ax,R2,bbox)
ax=plot_implicit(ax,R3,bbox)
ax=plot_implicit(ax,demand,bbox)

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …